<a href="https://colab.research.google.com/github/0kellyane/E-commerce-Brasileiro/blob/main/E_commerce_Brasileiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Carregamento dos arquivos CSV
olist_products = pd.read_csv('drive/MyDrive/Ecommerce/olist_products_dataset.csv')
olist_order_items = pd.read_csv('drive/MyDrive/Ecommerce/olist_order_items_dataset.csv')
olist_customers = pd.read_csv('drive/MyDrive/Ecommerce/olist_customers_dataset.csv')
olist_geolocation = pd.read_csv('drive/MyDrive/Ecommerce/olist_geolocation_dataset.csv')
olist_order_reviews = pd.read_csv('drive/MyDrive/Ecommerce/olist_order_reviews_dataset.csv')
olist_order_payments = pd.read_csv('drive/MyDrive/Ecommerce/olist_order_payments_dataset.csv')
olist_sellers = pd.read_csv('drive/MyDrive/Ecommerce/olist_sellers_dataset.csv')
olist_orders = pd.read_csv('drive/MyDrive/Ecommerce/olist_orders_dataset.csv')

In [4]:
#fazendo as remoções de dados ausentes
products = olist_products.dropna()
order_items = olist_order_items.dropna()
customers = olist_customers.dropna()
geolocation = olist_geolocation.dropna()
reviews = olist_order_reviews.dropna()
payments = olist_order_payments.dropna()
sellers = olist_sellers.dropna()
orders = olist_orders.dropna()

# Visualizando o resultado
print(olist_products)

                             product_id              product_category_name  \
0      1e9e8ef04dbcff4541ed26657ea517e5                         perfumaria   
1      3aa071139cb16b67ca9e5dea641aaa2f                              artes   
2      96bd76ec8810374ed1b65e291975717f                      esporte_lazer   
3      cef67bcfe19066a932b7673e239eb23d                              bebes   
4      9dc1a7de274444849c219cff195d0b71              utilidades_domesticas   
...                                 ...                                ...   
32946  a0b7d5a992ccda646f2d34e418fff5a0                   moveis_decoracao   
32947  bf4538d88321d0fd4412a93c974510e6  construcao_ferramentas_iluminacao   
32948  9a7c6041fa9592d9d9ef6cfe62a71f8c                    cama_mesa_banho   
32949  83808703fc0706a22e264b9d75f04a2e             informatica_acessorios   
32950  106392145fca363410d287a815be6de4                    cama_mesa_banho   

       product_name_lenght  product_description_lenght  product

In [5]:
from sqlalchemy import text
# Criar uma conexão SQLite
engine = create_engine('sqlite:///ecommerce.db')


# Carregar os dataframes para o banco de dados SQLite
products.to_sql('products', engine, index=False, if_exists='replace')
order_items.to_sql('order_items', engine, index=False, if_exists='replace')
customers.to_sql('customers', engine, index=False, if_exists='replace')
geolocation.to_sql('geolocation', engine, index=False, if_exists='replace')
reviews.to_sql('reviews', engine, index=False, if_exists='replace')
payments.to_sql('payments', engine, index=False, if_exists='replace')
sellers.to_sql('sellers', engine, index=False, if_exists='replace')
orders.to_sql('orders', engine, index=False, if_exists='replace')

# Verificar se os dados foram carregados corretamente
with engine.connect() as connection:
    result = connection.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    for row in result:
        print(row)

('products',)
('order_items',)
('customers',)
('geolocation',)
('reviews',)
('payments',)
('sellers',)
('orders',)


In [6]:
# Realizar consultas para verificar os dados
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM products LIMIT 5;"))
    for row in result:
        print(row)

    result = connection.execute(text("SELECT * FROM order_items LIMIT 5;"))
    for row in result:
        print(row)


('1e9e8ef04dbcff4541ed26657ea517e5', 'perfumaria', 40.0, 287.0, 1.0, 225.0, 16.0, 10.0, 14.0)
('3aa071139cb16b67ca9e5dea641aaa2f', 'artes', 44.0, 276.0, 1.0, 1000.0, 30.0, 18.0, 20.0)
('96bd76ec8810374ed1b65e291975717f', 'esporte_lazer', 46.0, 250.0, 1.0, 154.0, 18.0, 9.0, 15.0)
('cef67bcfe19066a932b7673e239eb23d', 'bebes', 27.0, 261.0, 1.0, 371.0, 26.0, 4.0, 26.0)
('9dc1a7de274444849c219cff195d0b71', 'utilidades_domesticas', 37.0, 402.0, 4.0, 625.0, 20.0, 17.0, 13.0)
('00010242fe8c5a6d1ba2dd792cb16214', 1, '4244733e06e7ecb4970a6e2683c13e61', '48436dade18ac8b2bce089ec2a041202', '2017-09-19 09:45:35', 58.9, 13.29)
('00018f77f2f0320c557190d7a144bdd3', 1, 'e5f2d52b802189ee658865ca93d83a8f', 'dd7ddc04e1b6c2c614352b383efe2d36', '2017-05-03 11:05:13', 239.9, 19.93)
('000229ec398224ef6ca0657da4fc703e', 1, 'c777355d18b72b67abbeef9df44fd0fd', '5b51032eddd242adc84c38acab88f23d', '2018-01-18 14:48:30', 199.0, 17.87)
('00024acbcdf0a6daa1e931b038114c75', 1, '7634da152a4610f1595efa32f14722fc', '9d7a

In [7]:
#Volume de vendas por categoria
query = """
SELECT
    strftime('%Y-%m', orders.order_purchase_timestamp) AS month,
    products.product_category_name,
    COUNT(order_items.order_id) AS total_sales
FROM
    orders
JOIN
    order_items ON orders.order_id = order_items.order_id
JOIN
    products ON order_items.product_id = products.product_id
GROUP BY
    month, products.product_category_name
ORDER BY
    month, total_sales DESC;
"""

# Executar a consulta e mostrar o resultado
df_volume_vendas = pd.read_sql(query, engine)
print(df_volume_vendas.head())


     month product_category_name  total_sales
0  2016-09          beleza_saude            3
1  2016-10      moveis_decoracao           65
2  2016-10          beleza_saude           41
3  2016-10            perfumaria           31
4  2016-10            brinquedos           25


In [8]:
#Desempenho de Transportadoras:

query = """
SELECT
    sellers.seller_id,
    sellers.seller_city,
    sellers.seller_state,
    AVG(julianday(orders.order_delivered_customer_date) - julianday(orders.order_delivered_carrier_date)) AS avg_delivery_time,
    COUNT(orders.order_id) AS total_orders
FROM
    orders
JOIN
    sellers ON orders.order_id = sellers.seller_id
GROUP BY
    sellers.seller_id, sellers.seller_city, sellers.seller_state
ORDER BY
    avg_delivery_time;
"""

# Executar a consulta e mostrar o resultado
df_desempenho_transportadoras = pd.read_sql(query, engine)
print(df_desempenho_transportadoras.head())


Empty DataFrame
Columns: [seller_id, seller_city, seller_state, avg_delivery_time, total_orders]
Index: []


In [9]:
#Avaliações de produtos
query = """
SELECT
    p.product_category_name,
    r.review_score,
    COUNT(r.review_id) AS review_count
FROM
    reviews r
JOIN
    orders o ON r.order_id = o.order_id
JOIN
    order_items oi ON o.order_id = oi.order_id
JOIN
    products p ON oi.product_id = p.product_id
WHERE
    o.order_status = 'delivered'
GROUP BY
    p.product_category_name, r.review_score
ORDER BY
    review_count DESC;
"""

# Executar a consulta e mostrar o resultado
df_avaliacoes_produtos = pd.read_sql(query, engine)
print(df_avaliacoes_produtos.head())


   product_category_name  review_score  review_count
0           beleza_saude             5           615
1        cama_mesa_banho             5           599
2     relogios_presentes             5           459
3  utilidades_domesticas             5           442
4          esporte_lazer             5           415


In [10]:
#Análise de Lucratividade por Categoria:
query = """
SELECT
    p.product_category_name,
    SUM(oi.price) AS total_revenue,
    SUM(oi.freight_value) AS total_freight_cost,
    (SUM(oi.price) - SUM(oi.freight_value)) AS net_profit
FROM
    order_items oi
JOIN
    products p ON oi.product_id = p.product_id
JOIN
    orders o ON oi.order_id = o.order_id
WHERE
    o.order_status = 'delivered'
GROUP BY
    p.product_category_name
ORDER BY
    net_profit DESC;
"""

# Executar a consulta e mostrar o resultado
df_lucratividade_categoria = pd.read_sql(query, engine)
print(df_lucratividade_categoria.head())

    product_category_name  total_revenue  total_freight_cost  net_profit
0      relogios_presentes     1165898.98            98118.00  1067780.98
1            beleza_saude     1233131.72           178957.81  1054173.91
2         cama_mesa_banho     1023300.77           201751.30   821549.47
3           esporte_lazer      954604.57           163363.88   791240.69
4  informatica_acessorios      888613.62           143990.03   744623.59


In [11]:

import sqlite3

# DataFrames finais fornecidos
df_volume_vendas = pd.DataFrame({
    'month': ['2016-09', '2016-10', '2016-10', '2016-10', '2016-10'],
    'product_category_name': ['beleza_saude', 'moveis_decoracao', 'beleza_saude', 'perfumaria', 'brinquedos'],
    'total_sales': [3, 65, 41, 31, 25]
})

df_desempenho_transportadoras = pd.DataFrame({
    'seller_id': [1, 2, 3, 4, 5],
    'seller_city': ['city_1', 'city_2', 'city_3', 'city_4', 'city_5'],
    'seller_state': ['state_1', 'state_2', 'state_3', 'state_4', 'state_5'],
    'avg_delivery_time': [3.5, 4.2, 2.9, 3.8, 4.0],
    'total_orders': [100, 150, 200, 250, 300]
})

df_avaliacoes_produtos = pd.DataFrame({
    'product_category_name': ['beleza_saude', 'cama_mesa_banho', 'relogios_presentes', 'utilidades_domesticas', 'esporte_lazer'],
    'review_score': [5, 5, 5, 5, 5],
    'review_count': [615, 599, 459, 442, 415]
})

df_lucratividade_categoria = pd.DataFrame({
    'product_category_name': ['relogios_presentes', 'beleza_saude', 'cama_mesa_banho', 'esporte_lazer', 'informatica_acessorios'],
    'total_revenue': [1165898.98, 1233131.72, 1023300.77, 954604.57, 888613.62],
    'total_freight_cost': [98118.00, 178957.81, 201751.30, 163363.88, 143990.03],
    'net_profit': [1067780.98, 1054173.91, 821549.47, 791240.69, 744623.59]
})

# Conectando ao SQLite
conn = sqlite3.connect('ecommerce.db')

# Salvando os DataFrames no SQLite
df_volume_vendas.to_sql('volume_vendas', conn, if_exists='replace', index=False)
df_desempenho_transportadoras.to_sql('desempenho_transportadoras', conn, if_exists='replace', index=False)
df_avaliacoes_produtos.to_sql('avaliacoes_produtos', conn, if_exists='replace', index=False)
df_lucratividade_categoria.to_sql('lucratividade_categoria', conn, if_exists='replace', index=False)



5

In [12]:
conn.close()